# Complete Code for NEMO Project

## 1. Activation of GPU

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-02-16 19:17:52.308417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 19:17:52.308439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 19:17:52.309180: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 19:17:52.313258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-16 19:17:52.864152: W tensorflow/compiler/tf2

Num GPUs Available:  1


2024-02-16 19:17:53.511374: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 19:17:53.532536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 19:17:53.532650: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## 2. Importing libraries

In [2]:
from ultralytics import YOLO
import cv2 
import time
import pandas as pd
from imutils import paths
import torch
import os
import csv

 You need to install pymongo>=3.9.0 in order to use MongoOutput 


## 3. Auxiliary Function (YOLO Load Models)

In [3]:
def load_yolo_models(model):
 
    models_dict = {
        'n': 'nano',
        's': 'small',
        'm': 'medium',
        'l': 'large',
        'x': 'extreme'
    }
 
    # Loading the three necessary models for code development.
    load_model = 'yolov8' + model
    model_people = YOLO('./weights/People/' + load_model + '.pt')
    model_rbnr = YOLO('./weights/BDBD/' + load_model + '.pt')
    model_svhn = YOLO('./weights/SVHN/' + load_model + '.pt')
    return model_people, model_rbnr, model_svhn

## 4. Auxiliary Function (People Detection)

In [4]:
def predict_people(model_people, image_path):
    """
    Perform prediction using the people model on the specified image.

    Parameters:
    - model_people: YOLO model for people prediction.
    - image_path: Path to the input image.

    Returns:
    - results_people: Results of the people prediction.
    """
    results_people = model_people.predict(source=image_path, classes=0)
    return results_people

## 5. Auxiliary Function (Processss People Results)

In [5]:
def process_results_people(results_people, image):
    """
    Process the results of people detection and extract cropped images.

    Parameters:
    - results_people: Results of people detection.
    - image: Original image.

    Returns:
    - cropped_images: List of cropped images based on the bounding box coordinates.
    """
    cropped_images = []

    # Iterate over each result in the list
    for result in results_people:
        boxes_people = result.boxes
        for coordenadas in boxes_people.xyxy:
            x_min, y_min, x_max, y_max = map(int, coordenadas)

            # Crop the image based on the bounding box coordinates
            cropped_image = image[y_min:y_max, x_min:x_max]

            # Store the cropped image in the list
            cropped_images.append(cropped_image)

    return cropped_images

## 6. Auxiliary Function (Bib Detection)

In [6]:
def predict_rbnr(model_rbnr, cropped_image):
    """
    Perform prediction using the RBNR model on the cropped image and process the results.

    Parameters:
    - model_rbnr: YOLO model for RBNR prediction.
    - cropped_image: Cropped image.

    Returns:
    - processed_rbnr: Processed data for the RBNR prediction.
      If prediction results are not empty, returns a dictionary with bounding box coordinates.
      Otherwise, returns None.
    """

    # Perform prediction using the RBNR model on the cropped image
    results_rbnr = model_rbnr.predict(source=cropped_image)[0]

    return results_rbnr

## 7. Auxiliary Function (Number Detection)

In [7]:
def get_svhn_number(image, model_svhn):
    """
    Gets the SVHN prediction number for a given image.

    Parameters:
    - image: Input image in OpenCV format (BGR).
    - model_svhn: YOLO model for SVHN prediction.

    Returns:
    - bib_number: The number extracted from the SVHN prediction.
    """

    # Perform prediction with the SVHN model for the image
    results_svhn = model_svhn.predict(source=image)[0]
    bib_list = []
    if results_svhn.boxes.xyxy.size(0) != 0:
        # Seleccionar el primer elemento de cada array
        primeros_elementos = results_svhn.boxes.xyxy[:, 0]
        # Obtener los índices ordenados de los primeros elementos
        indices_ordenados = torch.argsort(primeros_elementos)
        class_ordered = results_svhn.boxes.cls[indices_ordenados]
        for number in class_ordered:
            bib_list.append(str(int(number.item())))
        bib = "".join(bib_list)
        return bib
    else:
        return None
        

## 8. Auxiliary Function (CSV Writing)

In [8]:
def write_to_csv(csv_filename, fieldnames, bib_data):
    """
    Writes data to a CSV file.

    Parameters:
    - csv_filename: The name of the CSV file.
    - fieldnames: The field names for the CSV header.
    - bib_data: List of dictionaries containing data to be written to the CSV.
    """

    # Verify if the CSV file already exists
    csv_exists = os.path.isfile(csv_filename)

    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header only if the CSV file is new
        if not csv_exists:
            writer.writeheader()

        # Iterate over the dictionaries in bib_data and write to the CSV file
        for bib_entry in bib_data:
            
            # Write the entry to the CSV file
            writer.writerow(bib_entry)

## 9. Generic Code for the TGCRBNW Dataset

In [13]:
# Directory path containing the images.
dataset = 'TGCRBNWv0.2'
set = '5'
model = 's'
# Your path to the datasets you want to check
directory_path = './' + dataset + '/RP' + set + '_Subset'
# Get the list of files in the directory.
image_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.jpg')]
# We load the three necessary models for the code development.
model_people, model_rbnr, model_svhn = load_yolo_models(model)

# We create the list that will store all the information about the bibs.
bib_data = []
people_times = []
bib_times = []
number_times = []
# Iterate over each file in the directory.
start_time = time.time()
for image_file in image_files:
    # Build the full path of the image.
    image_path = os.path.join(directory_path, image_file)
    # We make our prediction for people.
    results_people = predict_people(model_people, image_path)
    # We load the image using OpenCV.
    image = cv2.imread(image_path)
    # Process the results of the list.
    cropped_images = process_results_people(results_people, image)
    # We iterate for each cropped image representing the people.
    for index, cropped_image in enumerate(cropped_images):
        # We make our prediction on the cropped image using the RBNR model.
        results_rbnr = predict_rbnr(model_rbnr, cropped_image)
        if results_rbnr.boxes.xyxy.size(0) != 0:
            x_min, y_min, x_max, y_max = map(int, results_rbnr.boxes.xyxy[0])
            # Crop the image based on coordinates
            new_image = cropped_image[y_min:y_max, x_min:x_max]
            # We make the number prediction on the new image.
            bib_number = get_svhn_number(new_image, model_svhn)
            bib_data.append({
                "image_name": image_file,
                "xmin": x_min,
                "ymin": y_min,
                "xmax": x_max,
                "ymax": y_max,
                "missing_label": bib_number
            })
end_time = time.time()

# Call the function to write to CSV
pred_csv_file = dataset + '_' + set + '_' + model + '.csv'
fieldnames = ['image_name', 'xmin', 'ymin', 'xmax', 'ymax', 'missing_label', 'notes']
write_to_csv(pred_csv_file, fieldnames, bib_data)
# Print times
print(f"El tiempo total para Set {set} en Yolov8{model} es de {end_time - start_time} segundos")


image 1/1 /home/gatvprojects/Desktop/NEMO/Datasets/TGCRBNWv0.2/RP5_Subset/RP5_frame_18_23_21_000.jpg: 384x640 1 person, 5.8ms
Speed: 1.9ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 768x320 1 bib, 4.6ms
Speed: 1.0ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 768, 320)

0: 672x768 1 2, 1 7, 1 9, 4.8ms
Speed: 1.6ms preprocess, 4.8ms inference, 1.0ms postprocess per image at shape (1, 3, 672, 768)

image 1/1 /home/gatvprojects/Desktop/NEMO/Datasets/TGCRBNWv0.2/RP5_Subset/RP5_frame_18_33_16_000.jpg: 384x640 1 person, 4.7ms
Speed: 1.3ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 768x320 1 bib, 4.4ms
Speed: 1.3ms preprocess, 4.4ms inference, 0.8ms postprocess per image at shape (1, 3, 768, 320)

0: 640x768 2 7s, 4.8ms
Speed: 1.6ms preprocess, 4.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 768)

image 1/1 /home/gatvprojects/Desktop/NEMO/Datasets/TGCRBNWv0.2

El tiempo total para Set 5 en Yolov8s es de 5.220515251159668 segundos


## 10.1 Calculation of Metrics for the TGCRBNW Dataset

In [18]:
# Directory path containing the images.
dataset = 'TGCRBNWv0.2'
set = '5'
model = 's'

# Path to the CSV file
csv_path = './' + dataset + '/RP' + set + '_etiquetado_v0.2.csv'

# Path to the folder containing the images
images_folder = './' + dataset + "/RP" + set + "_Subset"

# List all the image file names in the folder
image_files = os.listdir(images_folder)

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Filter the DataFrame to keep rows with image_name values corresponding to the image file names
subset_df = df[df['image_name'].isin(image_files)]

# Path to the CSV file containing the model predictions
predictions_csv_path = "./" + dataset + "/csv_generated_augmented/RP" + set + "_subset_csv/" + dataset + "_" + set + "_" + model + ".csv"

# Read the predictions CSV file into a DataFrame
predictions_df = pd.read_csv(predictions_csv_path)

# Conversion of the column missing_label into a int
TN = predictions_df['missing_label'].isna().sum()
predictions_df = predictions_df.dropna(subset=['missing_label'])
predictions_df['missing_label'] = predictions_df['missing_label'].astype(int)

# Initialize counts for true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN)
TP = 0
FP = 0
FN = 0


# Iterate through each row in the predictions DataFrame
for index, row in subset_df.iterrows():
    image_name = row['image_name']
    prediction = row['missing_label']

    # Check if the image name is present in the filtered DataFrame
    if image_name in predictions_df['image_name'].values:
        # Get the actual label from filtered_df based on image_name
        actual_labels = predictions_df.loc[predictions_df['image_name'] == image_name, 'missing_label'].values
        if prediction in actual_labels:
            TP += 1
        else:
            FP += 1
    else:
        FN += 1

# Print the counts
print("True Positives (TP):", TP)
print("True Negatives (TN):", TN )
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

True Positives (TP): 9
True Negatives (TN): 14
False Positives (FP): 47
False Negatives (FN): 18


## 11. Generic Code for the RBNR Dataset

In [21]:
# Directory path containing the images.
dataset = 'RBNR'
set = '3'
model = 's'
# Your path to the datasets you want to check
directory_path = './' + dataset + '/set' + set + '_org'
# Get the list of files in the directory.
image_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.JPG')]
# We load the three necessary models for the code development.
model_people, model_rbnr, model_svhn = load_yolo_models(model)

# We create the list that will store all the information about the bibs.
bib_data = []
people_times = []
bib_times = []
number_times = []
# Iterate over each file in the directory.
start_time = time.time()
# pyRAPL.setup(devices=None)
# meter = pyRAPL.Measurement('bar')
# meter.begin()
for image_file in image_files:
    # Build the full path of the image.
    image_path = os.path.join(directory_path, image_file)
    # We make our prediction for people.
    results_people = predict_people(model_people, image_path)
    # We load the image using OpenCV.
    image = cv2.imread(image_path)
    # Process the results of the list.
    cropped_images = process_results_people(results_people, image)
    # We iterate for each cropped image representing the people.
    for index, cropped_image in enumerate(cropped_images):
        # We make our prediction on the cropped image using the RBNR model.
        results_rbnr = predict_rbnr(model_rbnr, cropped_image)
        if results_rbnr.boxes.xyxy.size(0) != 0:
            x_min, y_min, x_max, y_max = map(int, results_rbnr.boxes.xyxy[0])
            # Crop the image based on coordinates
            new_image = cropped_image[y_min:y_max, x_min:x_max]
            # We make the number prediction on the new image.
            bib_number = get_svhn_number(new_image, model_svhn)
            bib_data.append({
                "image_name": image_file,
                "xmin": x_min,
                "ymin": y_min,
                "xmax": x_max,
                "ymax": y_max,
                "missing_label": bib_number
            })
end_time = time.time()
# meter.end()
# print(meter.result)
# Call the function to write to CSV
# pred_csv_file = 'RBNR1_bib_detection_yolov8x.csv'
pred_csv_file = dataset + '_' + set + '_' + model + '.csv'
fieldnames = ['image_name', 'xmin', 'ymin', 'xmax', 'ymax', 'missing_label', 'notes']
write_to_csv(pred_csv_file, fieldnames, bib_data)
# Print times
print(f"El tiempo total para Set {set} en Yolov8{model} es de {end_time - start_time} segundos")


image 1/1 /home/gatvprojects/Desktop/NEMO/Datasets/RBNR/set3_org/set3_20.JPG: 640x480 1 person, 86.1ms
Speed: 1.5ms preprocess, 86.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

0: 768x512 (no detections), 4.7ms
Speed: 1.7ms preprocess, 4.7ms inference, 0.5ms postprocess per image at shape (1, 3, 768, 512)

image 1/1 /home/gatvprojects/Desktop/NEMO/Datasets/RBNR/set3_org/set3_28.JPG: 640x480 5 persons, 4.7ms
Speed: 1.4ms preprocess, 4.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 768x320 1 bib, 5.1ms
Speed: 1.2ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 768, 320)

0: 768x736 1 3, 2 5s, 1 8, 1 9, 5.1ms
Speed: 1.9ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 736)

0: 768x224 (no detections), 5.0ms
Speed: 0.9ms preprocess, 5.0ms inference, 0.4ms postprocess per image at shape (1, 3, 768, 224)

0: 768x320 (no detections), 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 0.5m

El tiempo total para Set 3 en Yolov8s es de 6.485787868499756 segundos


## 11.1 Calculation of Metrics for the RBNR Dataset

In [24]:
# Directory path containing the images.
dataset = 'RBNR'
set = '3'
model = 's'

# Path to the CSV file
csv_path = './' + dataset + '/set' + set + '_org/list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Path to the CSV file containing the model predictions
predictions_csv_path = './' + dataset + '/set' + set + '_org_csv_augmented/' + dataset + "_" + set + "_" + model + ".csv"

# Read the predictions CSV file into a DataFrame
predictions_df = pd.read_csv(predictions_csv_path)

# Conversion of the column missing_label into a int
TN = predictions_df['missing_label'].isna().sum()
predictions_df = predictions_df.dropna(subset=['missing_label'])
predictions_df['missing_label'] = predictions_df['missing_label'].astype(int)

# Initialize counts for true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN)
TP = 0
FP = 0
FN = 0


# Iterate through each row in the predictions DataFrame
for index, row in df.iterrows():
    image_name = row['Image']
    prediction = row['RBN']

    # Check if the image name is present in the filtered DataFrame
    if image_name in predictions_df['image_name'].values:
        # Get the actual label from filtered_df based on image_name
        actual_labels = predictions_df.loc[predictions_df['image_name'] == image_name, 'missing_label'].values
        if prediction in actual_labels:
            TP += 1
        else:
            FP += 1
    else:
        FN += 1
        
# Print the counts
print("True Positives (TP):", TP)
print("True Negatives (TN):", TN )
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

True Positives (TP): 51
True Negatives (TN): 5
False Positives (FP): 58
False Negatives (FN): 4
